In [1]:
%matplotlib tk
import pandas
import pafy
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter, MultipleLocator


In [2]:
master_df = pandas.DataFrame(
    columns=['seconds', 'timer_value', 'idx']
).set_index('seconds')

In [3]:
def ts_to_int(s):
    hours, minutes, seconds = s.split(':')
    return 3600 * int(hours) + 60 * int(minutes) + int(seconds)

In [4]:
def seconds_to_string(n: int, _) -> str:
    hour = int(n // 3600)
    seconds = int(n % 60)
    minutes = int((n // 60) % 60)
    return f"{hour}:{minutes:02}:{seconds:02}"

In [5]:
elapsed_time = 0
for fp in sorted(os.listdir('csvs/cleaned')):
    try:
        idx, qstring = fp.split('|')
    except ValueError:
        continue
    qstring = qstring.replace('.csv', '')
    url = f'https://www.youtube.com/watch?v={qstring}'

    df = pandas.read_csv('csvs/cleaned/' + fp)
    df['seconds'] = df['video_timestamp'].apply(ts_to_int)
    df['seconds'] += elapsed_time
    df.set_index('seconds', inplace=True)
    df['idx'] = idx
    
    master_df = pandas.concat( [master_df, df.loc[:, ['timer_value', 'idx']] ] )

    
    elapsed_time += pafy.new(url).length

In [6]:
master_df

,timer_value,idx
seconds,,
1,216186,14
2,216185,14
3,216184,14
4,216183,14
5,216182,14
...,...,...
636871,65631,31
636872,65630,31
636873,65629,31


In [7]:
master_df.to_csv('csvs/cleaned/all.csv')

In [8]:

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(master_df.index, master_df['timer_value'], markersize=1)

ax.xaxis.set_major_formatter(FuncFormatter(seconds_to_string))
ax.xaxis.set_major_locator(MultipleLocator(base=3600*24))
ax.set_xlabel('time into stream')

ax.yaxis.set_major_formatter(FuncFormatter(seconds_to_string))
ax.yaxis.set_major_locator(MultipleLocator(base=3600*12))
ax.set_ylim([0, None])
ax.set_ylabel('time remaining')
ax.set_title(f'All VODs')
fig.text(0.6, 0.9, f'Made by u/2435191 with cv2, tesseract, pandas, and mpl')
fig.savefig(f'graphs/all.png', dpi=400)